This module presents an approach for assessing change in built-up land cover in Trikala and Volos,  Greece. 

In [1]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal

try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

In [2]:
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

In [3]:
magnisia = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM2_NAME','Magnisias')).geometry()
trikala = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM2_NAME','Trikkaion')).geometry()


ModuleNotFoundError: No module named 'h3pandas'

In [4]:


# get our Nepal boundary
thessalia = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM1_NAME','Thessalia')).geometry()

# Sentinel-2 image filtered on 2019 and on Nepal
se2 = ee.ImageCollection('COPERNICUS/S2').filterDate("2021-01-01","2022-01-01").filterBounds(thessalia).median().divide(10000)

rgb = ['B4','B3','B2']

# set some thresholds
rgbViz = {"min":0.0, "max":0.3,"bands":rgb}


# initialize our map
map1 = geemap.Map()
map1.centerObject(thessalia, 7)
map1.addLayer(se2.clip(thessalia), rgbViz, "S2")

map1.addLayerControl()
map1

Map(center=[39.51314304598321, 22.211757970370197], controls=(WidgetControl(options=['position', 'transparent_…

In [5]:


# get our Nepal boundary

# Sentinel-2 image filtered on 2019 and on Nepal
se2 = ee.ImageCollection('COPERNICUS/S2').filterDate("2015-01-01","2022-05-01").filterBounds(magnisia).median().divide(10000)

rgb = ['B4','B3','B2']

# set some thresholds
rgbViz = {"min":0.0, "max":0.3,"bands":rgb}


# initialize our map
map1 = geemap.Map()
map1.centerObject(magnisia, 7)
map1.addLayer(se2.clip(magnisia), rgbViz, "S2")

map1.addLayerControl()
map1

Map(center=[39.249507776111805, 22.970044299390757], controls=(WidgetControl(options=['position', 'transparent…

We can see a real color image of Magnisia. We reduced our Image Collection to an image representing the median of 2021 and it appears we’ve also captured some clouds around Kathmandu. We will make a cloud mask to clear the image up using Sentinel-2’s QA band. We’re modeling this (in Python) from the example used in GEE: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2#bands

In [6]:
def se2mask(image):
    quality_band = image.select('QA60')
    
    # using the bit mask for clouds and cirrus clouds respectively
    cloudmask = 1 << 10
    cirrusmask = 1 << 11
    
    # we only want clear skies
    mask = quality_band.bitwiseAnd(cloudmask).eq(0) and (quality_band.bitwiseAnd(cirrusmask).eq(0))
    
    # we'll divide by 10000 to make interpreting the reflectance values easier
    return image.updateMask(mask).divide(10000)
    
se2 = ee.ImageCollection('COPERNICUS/S2').filterDate(
    "2021-01-01","2022-01-01").filterBounds(magnisia).filter(
    ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",20)).map(se2mask).median()

# initialize our map
map2 = geemap.Map()
map2.centerObject(magnisia, 7)
map2.addLayer(se2.clip(magnisia), rgbViz, "S2")

map2.addLayerControl()
map2

Map(center=[39.249507776111805, 22.970044299390757], controls=(WidgetControl(options=['position', 'transparent…

Next we'll integrate Sentinel-2 image data with VIIRS-DNB nighttime lights.



Intro to Global Human Settlement Layer¶
Identifying areas of human settlement is a large area of focus in Earth Observation and many other disciplines.

The Global Human Settlement Layer (GHSL) dataset is a useful resource for understanding areas of settlement and happily for our purposes, it is available in the Google Earth Engine (GEE) catalogue.

The dataset we are particularly interested in is the GHSL “Settlement Grid” layer. The settlement grids in this dataset have been generated via the GHSL built-up areas and population grids, which are themselves derived from Landsat image collections and other sources and these layers are also available on GEE.

More details and links to resources are available on the collection’s landing pages, including a description and link to the methodology.

A couple things to note.

First, there is one band with four “degrees of urbanization”:

Inhabited areas (

Rural grid cells

Low Density Clusters (towns and cities)

High Density Clusters (cities)

We made a choice in framing our analysis that we are interested in the change of the Low and High Density clusters (“built up”) relative to everything else, so we will classify any pixel with values in [3,4] as “built up” and assign this 1 or not and assign it 0.

A second item worth noting is that the spatial resolution for this grid layer is 1000 meters.

Let’s take a quick look at the data…

In [33]:
#2 dw

In [13]:
Map = geemap.Map()
Map.add_basemap('HYBRID')

# # Set the region of interest by simply drawing a polygon on the map
# region = Map.user_roi
# if region is None:
#     region = ee.Geometry.BBox(-89.7088, 42.9006, -89.0647, 43.2167)

Map.centerObject(magnisia)
# Set the date range
start_date = '2021-01-01'
end_date = '2022-01-01'

# Create a Sentinel-2 image composite
image = geemap.dynamic_world_s2(magnisia, start_date, end_date)
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
Map.addLayer(image, vis_params, 'Sentinel-2 image')

# Create Dynamic World land cover composite
landcover = geemap.dynamic_world(magnisia, start_date, end_date, return_type='hillshade')
Map.addLayer(landcover, {}, 'Land Cover')

# Add legend to the map
Map.add_legend(title="Dynamic World Land Cover", builtin_legend='Dynamic_World')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In the original data layer ("Degree of Urbanization") you can see the high density patch of Kathmandu (white) as well as the low density cluster classifications that follow what appears to be major road networks and other towns (bright green). We can also see the rural areas (green) and the rest of the country (black).

In our binary mask ("Built up"), you can see how the coverage is over both low and high cluster areas. This is the layer we will use as our labels for the training data.

In [7]:
# our Region Of Interest is the Province of Bagmati
#roi = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM2_NAME','Bagmati')).geometry()

def se2mask(image):
    quality_band = image.select('QA60')
    cloudmask = 1 << 10
    cirrusmask = 1 << 11
    mask = quality_band.bitwiseAnd(cloudmask).eq(0) and (quality_band.bitwiseAnd(cirrusmask).eq(0))
    return image.updateMask(mask).divide(10000)
    
se2 = ee.ImageCollection('COPERNICUS/S2').filterDate(
    "2019-01-01","2019-12-31").filterBounds(magnisia).filter(
    ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",20)).map(se2mask).median().clip(magnisia)

viirs = ee.Image(ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2019-01-01","2019-12-31").filterBounds(magnisia).median().select('avg_rad').clip(magnisia))

ghsl = ee.ImageCollection('JRC/GHSL/P2016/SMOD_POP_GLOBE_V1').filter(ee.Filter.date('2015-01-01', '2015-12-31')).select('smod_code').median().clip(magnisia)

ghsl = ghsl.gte(2)

ghslVis= {"palette":['000000', 'ffffff']}
se2Vis = {"min":0.0, "max":0.3,"bands": ['B4','B3','B2']}

# initialize our map
map1 = geemap.Map()
map1.centerObject(magnisia, 9)
map1.addLayer(se2, se2Vis, "S2")
map1.addLayer(viirs, {}, "VIIRS-DNB", opacity=0.5)
map1.addLayer(ghsl, ghslVis, "GHSL", opacity=0.25)
map1.addLayerControl()
map1

Map(center=[39.249507776111805, 22.970044299390757], controls=(WidgetControl(options=['position', 'transparent…

In [11]:
viirs2015 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(
    "2016-01-01","2016-12-31").filterBounds(magnisia).median().select('avg_rad').clip(magnisia)
viirs2019 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(
    "2021-01-01","2021-12-31").filterBounds(magnisia).median().select('avg_rad').clip(magnisia)

viirs_15_tile = geemap.ee_tile_layer(viirs2015, {}, 'Jul-Dec 2015', opacity=0.9)
viirs_22_tile = geemap.ee_tile_layer(viirs2019, {}, 'Jul-Dec 2022', opacity=0.9)

# initialize our map
map2 = geemap.Map(basemap='HYBRID')
map2.centerObject(magnisia, 9)
map2.split_map(left_layer=viirs_15_tile, right_layer=viirs_22_tile)
map2.addLayerControl()
map2

Map(center=[39.249507776111805, 22.970044299390757], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [12]:
mu15 = viirs2015.reduceRegion(reducer=ee.Reducer.mean(),scale=500)
std15 = viirs2015.reduceRegion(reducer=ee.Reducer.stdDev(),scale=500)
mu19 = viirs2019.reduceRegion(reducer=ee.Reducer.mean(),scale=500)
std19 = viirs2019.reduceRegion(reducer=ee.Reducer.stdDev(),scale=500)

# we'll cast these to native ee Numbers using the ee.Number constructor
mu15 = ee.Number(mu15.get('avg_rad'))
std15 = ee.Number(std15.get('avg_rad'))
mu19 = ee.Number(mu19.get('avg_rad'))
std19 = ee.Number(std19.get('avg_rad'))

viirs2015 = viirs2015.subtract(mu15).divide(std19)
viirs2019 = viirs2019.subtract(mu15).divide(std19)

viirs_15_tile = geemap.ee_tile_layer(viirs2015, {}, 'Jul-Dec 2015', opacity=0.9)
viirs_19_tile = geemap.ee_tile_layer(viirs2019, {}, 'Jul-Dec 2019', opacity=0.9)

# initialize our map
map3 = geemap.Map(basemap='HYBRID')
map3.centerObject(magnisia, 9)
map3.split_map(left_layer=viirs_15_tile, right_layer=viirs_19_tile)
map3.addLayerControl()
map3

Map(center=[39.249507776111805, 22.970044299390757], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [ ]:
import h3pandas
h3=geojson2h3.featureToH3Set(gdf)
h3.h3_to_geo_boundary(gdf, geo_json=False)
import googlemaps
h3.polyfill(gdf, 9, geo_json_conformant=False)

In [ ]:
import h3
from shapely.geometry import Polygon, Point
import shapely.wkt

def output_h3_id_attributes(h3_id):
    return {
        "co_ordinates" : h3.h3_to_geo(h3_id), 
        "geo_boundary" : Polygon(h3.h3_to_geo_boundary(h3_id, geo_json=True)).wkt, 
        "parent" : h3.h3_to_parent(h3_id), 
        "children" : h3.h3_to_children(h3_id)
    }
output_h3_id_attributes('8843acd819fffff')
